In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,cohen_kappa_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None 

In [ ]:
def train_test_data_split(raw_data, train_mapping, test_mapping):
    train_data = pd.DataFrame(columns=raw_data.columns)
    test_data = pd.DataFrame(columns=raw_data.columns)
    
    for case_id in raw_data.index:
        case_general_id = case_id[0:20]
        if case_general_id in train_mapping.values[:,1]:
            train_data.loc[case_id]= raw_data.loc[case_id, :]
        elif case_general_id in test_mapping.values[:,1]:
            test_data.loc[case_id]= raw_data.loc[case_id, :]
            
    return train_data, test_data

In [ ]:
def select_features(dataset, number_of_features):
    X = dataset.loc[:, dataset.columns != "Grade"]
    Y = dataset.loc[:, "Grade"]
    
    standard_scaler = StandardScaler().fit(X)
    X[X.columns] = standard_scaler.fit_transform(X[X.columns])
    
    select_k_best = SelectKBest(f_classif).fit(X, Y)
    
    scored_features = pd.DataFrame({'Feature':list(X.columns), 'Score':select_k_best.scores_})
    scored_features.sort_values(by='Score', ascending=False, inplace=True)
    scored_features.to_csv("output/scored_features.csv")
    
    best_features = scored_features.nlargest(number_of_features,'Score')
    best_features = best_features.loc[:, "Feature"]
    # print("Selected features: ")
    # print(best_features)
    best_features.to_csv("output/best_features.csv")

    selected_features = X.loc[dataset.index, best_features]
    selected_features["Grade"] = Y
    
    return selected_features

In [ ]:
def split_modalities(input_data):
    
    flair= pd.DataFrame(columns=input_data.columns)
    t1c = pd.DataFrame(columns=input_data.columns)
    t1= pd.DataFrame(columns=input_data.columns)
    t2 = pd.DataFrame(columns=input_data.columns)
    
    for case_id in input_data.index:
        if "FLAIR" in case_id:
            flair.loc[case_id]= input_data.loc[case_id, :]
        elif "T1C" in case_id:
            t1c.loc[case_id]= input_data.loc[case_id, :]
        elif "T1" in case_id:
            t1.loc[case_id]= input_data.loc[case_id, :]
        elif "T2" in case_id:
            t2.loc[case_id]= input_data.loc[case_id, :]
            
            
    labels = (flair.loc[:, 'Grade']).to_numpy()
    
    flair.drop('Grade',axis='columns', inplace=True)
    t1c.drop('Grade',axis='columns', inplace=True)
    t1.drop('Grade',axis='columns', inplace=True)
    t2.drop('Grade',axis='columns', inplace=True)
    
    return flair, t1c, t1, t2, labels

In [ ]:
def preprocess_test_dataset(dataset):
    selected_features = pd.read_csv("output/best_features.csv", index_col=0)
    features_list = selected_features["Feature"].to_list()
    
    X = dataset.loc[:, dataset.columns != "Grade"]
    Y = dataset.loc[:, "Grade"]
    
    reduced_X = X.loc[:, features_list]
    
    standard_scaler = StandardScaler().fit(X)
    reduced_X[reduced_X.columns] = standard_scaler.fit_transform(reduced_X[reduced_X.columns])
    
    reduced_dataset = reduced_X
    reduced_dataset["Grade"] = Y
    
    return reduced_dataset

In [ ]:
def checkMajority(voting_arr):
    lgg_occurences = voting_arr.count("LGG")
    hgg_occurences = voting_arr.count("HGG")
    if  hgg_occurences >= lgg_occurences:
        return "HGG"
    else:
        return "LGG"

In [ ]:
dataset = pd.read_csv("output/features.csv", index_col=0)
name_mapping = pd.read_csv("mapping/name_mapping.csv")

skf = StratifiedKFold(n_splits=5)

X = name_mapping.values[:,1]
y = name_mapping.values[:,0]
fold_iterator = 1

overall_acc = []
overall_kappa = []
overall_mcc = []

number_of_features = 6

for train_index, test_index in skf.split(X, y):
    print("\n")
    print("FOLD NUMBER: ", fold_iterator)
    print("\n")
    fold_iterator += 1
     
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    # print("Train labels distribution:", Counter(y_train))
    # print("Test labels distribution:",Counter(y_test))
    # print("\n")

    train_mapping = pd.DataFrame({'Grade': y_train, 'ID': X_train})    
    test_mapping = pd.DataFrame({'Grade': y_test, 'ID': X_test})
    
    (train_dataset, test_dataset) = train_test_data_split(dataset, train_mapping, test_mapping)
    
    reduced_train_dataset = select_features(train_dataset, number_of_features)
    
    (flair_data, t1c_data, t1_data, t2_data, labels) = split_modalities(reduced_train_dataset)
    
    flair_classifier = RandomForestClassifier()
    t1c_classifier = RandomForestClassifier()
    t1_classifier = RandomForestClassifier()
    t2_classifier = RandomForestClassifier()
    
    flair_classifier.fit(flair_data.to_numpy(), labels)
    t1c_classifier.fit(t1c_data.to_numpy(), labels)
    t1_classifier.fit(t1_data.to_numpy(), labels)
    t2_classifier.fit(t2_data.to_numpy(), labels)
    
    print("Classifiers training finished")

    preprocessed_test_dataset = preprocess_test_dataset(test_dataset)
    
    (test_flair_data, test_t1c_data, test_t1_data, test_t2_data, test_labels) = split_modalities(preprocessed_test_dataset)
    
    number_of_test_cases = len(test_mapping)
    
    test_results = []

    for i in range(number_of_test_cases):
        voting_arr = []

        flair_result = flair_classifier.predict(test_flair_data.iloc[i].to_numpy().reshape(1,-1))[0]
        voting_arr.append(flair_result)

        t1c_result = t1c_classifier.predict(test_t1c_data.iloc[i].to_numpy().reshape(1,-1))[0]
        voting_arr.append(t1c_result)

        t1_result = t1_classifier.predict(test_t1_data.iloc[i].to_numpy().reshape(1,-1))[0]
        voting_arr.append(t1_result)

        t2_result = t2_classifier.predict(test_t2_data.iloc[i].to_numpy().reshape(1,-1))[0]
        voting_arr.append(t2_result)

        final_prediction = checkMajority(voting_arr)

        test_results.append(final_prediction)
        
    test_results = np.array(test_results)
    
    print(confusion_matrix(test_labels, test_results))
    # print(classification_report(test_labels, test_results))
    
    acc = accuracy_score(test_labels, test_results)
    kappa = cohen_kappa_score(test_labels, test_results)
    mcc = matthews_corrcoef(test_labels, test_results)
    
    print("\n")
    print("Accuracy: {:.5f}".format(acc))
    print("Cohen's Kappa: {:.5f}".format(kappa))
    print("MCC: {:.5f}".format(mcc))
    
    overall_acc.append(acc)
    overall_kappa.append(kappa)
    overall_mcc.append(mcc)  
    
general_accuracy = pd.DataFrame({'Accuracy': overall_acc})
general_kappa = pd.DataFrame({'Kappa': overall_kappa})
general_mcc = pd.DataFrame({'MCC': overall_mcc})

accuracy_mean = float(general_accuracy.mean())
accuracy_median = float(general_accuracy.median())
accuracy_Q1 = general_accuracy.Accuracy.quantile([0.25]).to_numpy()[0]
accuracy_Q3 = general_accuracy.Accuracy.quantile([0.75]).to_numpy()[0]
accuracy_IQR = accuracy_Q3 - accuracy_Q1


kappa_mean = float(general_kappa.mean())
kappa_median = float(general_kappa.median())
kappa_Q1 = general_kappa.Kappa.quantile([0.25]).to_numpy()[0]
kappa_Q3 = general_kappa.Kappa.quantile([0.75]).to_numpy()[0]
kappa_IQR = kappa_Q3 - kappa_Q1

mcc_mean = float(general_mcc.mean())
mcc_median = float(general_mcc.median())
mcc_Q1 = general_mcc.MCC.quantile([0.25]).to_numpy()[0]
mcc_Q3 = general_mcc.MCC.quantile([0.75]).to_numpy()[0]
mcc_IQR = mcc_Q3 - mcc_Q1

accuracy_list = [accuracy_mean, accuracy_median, accuracy_Q1, accuracy_Q3, accuracy_IQR]
kappa_list = [kappa_mean, kappa_median, kappa_Q1, kappa_Q3, kappa_IQR]
mcc_list = [mcc_mean, mcc_median, mcc_Q1, mcc_Q3, mcc_IQR]

rows_labels = ["Mean", "Median", "Q1", "Q2", "IQR"]
column_labels = ["Accuracy", "Kappa", "MCC"]
metrics = pd.DataFrame(index=rows_labels, columns=column_labels)

metrics["Accuracy"] = acc_list
metrics["Kappa"] = kappa_list
metrics["MCC"] = mcc_list

rounded_metrics = metrics.round(3)
rounded_metrics.to_csv("metrics.csv")

In [ ]:
rounded_metrics

In [ ]:
metics_dict = {}
nof = []
for number_of_features in range(42,49):
    print("FEATURES NUMBER: ", number_of_features)
 
    nof.append(number_of_features)
    
    dataset = pd.read_csv("output/features.csv", index_col=0)
    name_mapping = pd.read_csv("mapping/name_mapping.csv")

    skf = StratifiedKFold(n_splits=5)

    X = name_mapping.values[:,1]
    y = name_mapping.values[:,0]
    fold_iterator = 1

    overall_acc = []
    overall_kappa = []
    overall_mcc = []

    for train_index, test_index in skf.split(X, y):
        print("FOLD NUMBER: ", fold_iterator)
        fold_iterator += 1

        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]

        # print("Train labels distribution:", Counter(y_train))
        # print("Test labels distribution:",Counter(y_test))
        # print("\n")

        train_mapping = pd.DataFrame({'Grade': y_train, 'ID': X_train})    
        test_mapping = pd.DataFrame({'Grade': y_test, 'ID': X_test})

        (train_dataset, test_dataset) = train_test_data_split(dataset, train_mapping, test_mapping)

        reduced_train_dataset = select_features(train_dataset, number_of_features)

        (flair_data, t1c_data, t1_data, t2_data, labels) = split_modalities(reduced_train_dataset)

        flair_classifier = RandomForestClassifier()
        t1c_classifier = RandomForestClassifier()
        t1_classifier = RandomForestClassifier()
        t2_classifier = RandomForestClassifier()

        flair_classifier.fit(flair_data.to_numpy(), labels)
        t1c_classifier.fit(t1c_data.to_numpy(), labels)
        t1_classifier.fit(t1_data.to_numpy(), labels)
        t2_classifier.fit(t2_data.to_numpy(), labels)

        preprocessed_test_dataset = preprocess_test_dataset(test_dataset)

        (test_flair_data, test_t1c_data, test_t1_data, test_t2_data, test_labels) = split_modalities(preprocessed_test_dataset)

        number_of_test_cases = len(test_mapping)

        test_results = []

        for i in range(number_of_test_cases):
            voting_arr = []

            flair_result = flair_classifier.predict(test_flair_data.iloc[i].to_numpy().reshape(1,-1))[0]
            voting_arr.append(flair_result)

            t1c_result = t1c_classifier.predict(test_t1c_data.iloc[i].to_numpy().reshape(1,-1))[0]
            voting_arr.append(t1c_result)

            t1_result = t1_classifier.predict(test_t1_data.iloc[i].to_numpy().reshape(1,-1))[0]
            voting_arr.append(t1_result)

            t2_result = t2_classifier.predict(test_t2_data.iloc[i].to_numpy().reshape(1,-1))[0]
            voting_arr.append(t2_result)

            final_prediction = checkMajority(voting_arr)

            test_results.append(final_prediction)

        test_results = np.array(test_results)

        # print(confusion_matrix(test_labels, test_results))
        # print(classification_report(test_labels, test_results))

        acc = accuracy_score(test_labels, test_results)
        kappa = cohen_kappa_score(test_labels, test_results)
        mcc = matthews_corrcoef(test_labels, test_results)

        # print("\n")
        # print("Accuracy: {:.5f}".format(acc))
        # print("Cohen's Kappa: {:.5f}".format(kappa))
        # print("MCC: {:.5f}".format(mcc))

        overall_acc.append(acc)
        overall_kappa.append(kappa)
        overall_mcc.append(mcc)  

    general_accuracy = pd.DataFrame({'Accuracy': overall_acc})
    general_kappa = pd.DataFrame({'Kappa': overall_kappa})
    general_mcc = pd.DataFrame({'MCC': overall_mcc})

    accuracy_mean = float(general_accuracy.mean())
    accuracy_median = float(general_accuracy.median())
    accuracy_Q1 = general_accuracy.Accuracy.quantile([0.25]).to_numpy()[0]
    accuracy_Q3 = general_accuracy.Accuracy.quantile([0.75]).to_numpy()[0]
    accuracy_IQR = accuracy_Q3 - accuracy_Q1


    kappa_mean = float(general_kappa.mean())
    kappa_median = float(general_kappa.median())
    kappa_Q1 = general_kappa.Kappa.quantile([0.25]).to_numpy()[0]
    kappa_Q3 = general_kappa.Kappa.quantile([0.75]).to_numpy()[0]
    kappa_IQR = kappa_Q3 - kappa_Q1

    mcc_mean = float(general_mcc.mean())
    mcc_median = float(general_mcc.median())
    mcc_Q1 = general_mcc.MCC.quantile([0.25]).to_numpy()[0]
    mcc_Q3 = general_mcc.MCC.quantile([0.75]).to_numpy()[0]
    mcc_IQR = mcc_Q3 - mcc_Q1

    accuracy_list = [accuracy_mean, accuracy_median, accuracy_Q1, accuracy_Q3, accuracy_IQR]
    kappa_list = [kappa_mean, kappa_median, kappa_Q1, kappa_Q3, kappa_IQR]
    mcc_list = [mcc_mean, mcc_median, mcc_Q1, mcc_Q3, mcc_IQR]

    rows_labels = ["Mean", "Median", "Q1", "Q2", "IQR"]
    column_labels = ["Accuracy", "Kappa", "MCC"]
    metrics = pd.DataFrame(index=rows_labels, columns=column_labels)

    metrics["Accuracy"] = accuracy_list
    metrics["Kappa"] = kappa_list
    metrics["MCC"] = mcc_list

    rounded_metrics = metrics.round(3)
    metics_dict[number_of_features] = rounded_metrics

In [ ]:
accuracy_mean = []
kappa_mean = []
mcc_mean = [] 

accuracy_median = []
kappa_median = []
mcc_median = []

for i in nof:
    accuracy_mean.append(metics_dict[i]["Accuracy"]["Mean"])
    kappa_mean.append(metics_dict[i]["Kappa"]["Mean"])
    mcc_mean.append(metics_dict[i]["MCC"]["Mean"])
    
    accuracy_median.append(metics_dict[i]["Accuracy"]["Median"])
    kappa_median.append(metics_dict[i]["Kappa"]["Median"])
    mcc_median.append(metics_dict[i]["MCC"]["Median"])

In [ ]:
plt.plot(nof, kappa_mean)
plt.plot(nof, mcc_mean)
plt.legend(["Kappa", "MCC"])

In [ ]:
plt.plot(nof, accuracy_mean)
plt.legend(["Accuracy"])

In [ ]:
plt.plot(nof, kappa_median)
plt.plot(nof, mcc_median)
plt.legend(["Kappa", "MCC"])

In [ ]:
plt.plot(nof, accuracy_median)
plt.legend(["Accuracy"])